In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import daglet
import operator
from functools import reduce
from fuzix.draw_utils import draw_line
from fuzix.draw_utils import draw_circle
from ipywidgets import interact

In [ ]:
class StateSymbol(object):
    def __init__(self, name='state'):
        self.name = name

    def __repr__(self):
        return '{}<{:x}>'.format(self.name, hash(self) % (1<<32))
    

class Connector(object):
    def __init__(self, spec=None, parents=[]):
        self.spec = spec
        self.parents = parents

    def __or__(self, other):
        if self.spec is not None:
            parents = [self]
        else:
            parents = []
        return other.connector_cls(other, parents)

    def draw(self, ax, params, state):
        return self.spec.draw(self, ax, params, state)


class Spec(object):
    def __init__(self, state_symbols):
        self.state_symbols = state_symbols

    connector_cls = Connector


class Pendulum(Spec):
    def __init__(self, length=1., mass=1.):
        self.theta = StateSymbol('theta')
        super(Pendulum, self).__init__([self.theta])
        self.length = length
        self.mass = mass

    def get_accel_matrix(self, state_map):
        theta, dtheta = state_map[self.theta]
        return self.length * np.matrix([
            [
                np.cos(theta),
                dtheta**2 * -np.sin(theta),
            ],
            [
                np.sin(theta),
                dtheta**2 * np.cos(theta),
            ],
        ])
    
    def get_perturbation(self, state_map, state_symbol):
        if state_symbol is self.theta:
            theta, dtheta = state_map[self.theta]
            return self.length * np.matrix([
                [np.cos(theta)],
                [np.sin(theta)],
            ])
        else:
            raise ValueError('unknown symbol: {}'.format(state_symbol))

    
class Scene(object):
    def _get_transitive_children(self, connector):
        return set(daglet.toposort([connector], lambda x: self.child_map.get(x, set())))
    
    def _get_transitive_parents(self, connector):
        return set(daglet.toposort([connector], lambda x: x.parents))

    def _get_transitive_mass(self, connector):
        return sum(x.spec.mass for x in self.transitive_child_map[connector])
    
    def _get_pert_influence(self, pert_connector):
        return (
            self.transitive_child_map[pert_connector]
            .union(self.transitive_parent_map[pert_connector])            
        )

    def __init__(self, connectors):
        self.connectors = connectors
        self.connectors = daglet.toposort(connectors, lambda x: x.parents)
        self.specs = [x.spec for x in self.connectors]
        #self.state_symbol_map = {}
        #for connector in self.connectors:
        #    for state_symbol in connector.spec.state_symbols:
        #        self.state_symbol_map[state_symbol] = connector
        #self.state_symbols = list(self.state_symbol_map.keys())
        self.state_symbols = reduce(operator.add, [x.state_symbols for x in self.specs])
        self.symbol_id_map = {symbol: i for i, symbol in enumerate(self.state_symbols)}
        self.child_map = daglet.get_child_map(connectors, lambda x: x.parents)
        self.transitive_child_map = {x: self._get_transitive_children(x) for x in self.connectors}
        self.transitive_parent_map = {x: self._get_transitive_parents(x) for x in self.connectors}
        self.transitive_mass_map = {x: self._get_transitive_mass(x) for x in self.connectors}
        self.pert_influence_map = {x: self._get_pert_influence(x) for x in self.connectors}

    def _rank_connector(self, connector):
        return len(self.transitive_parent_map[connector])
    
    def _get_max_rank_connector(self, *connectors):
        return max(connectors, key=self._rank_connector)

    def get_pert_accel_coefficient(self, pert_connector, accel_connector):
        max_rank_connector = scene._get_max_rank_connector(pert_connector, accel_connector)
        return scene.transitive_mass_map[max_rank_connector]


def tick(scene, state_map, dt=0.01):
    rows = []
    accel_matrix_map = {x: x.spec.get_accel_matrix(state_map) for x in scene.connectors}
    for pert_connector in scene.connectors:
        for pert_symbol in pert_connector.spec.state_symbols:
            perturbation = pert_connector.spec.get_perturbation(state_map, pert_symbol)
            remainder = 0.
            accel_map = [None] * len(scene.symbol_id_map)
            for accel_connector in scene.pert_influence_map[pert_connector]:
                coefficient = scene.get_pert_accel_coefficient(pert_connector, accel_connector)
                accel_matrix = accel_matrix_map[accel_connector]
                for col, accel_symbol in enumerate(accel_connector.spec.state_symbols):
                    accel_symbol_id = scene.symbol_id_map[accel_symbol]
                    accel_map[accel_symbol_id] = float(coefficient * accel_matrix[:,col].T * perturbation)
                remainder += float(coefficient * accel_matrix[:,-1].T * perturbation)
            rows.append(accel_map + [remainder])
    m = np.matrix(rows)
    A = m[:,:-1]
    b = -m[:,-1]
    ddq = np.linalg.inv(A) * b
    ddq = {scene.state_symbols[i]: float(v) for i, v in enumerate(ddq)}
    new_state_map = {}
    for symbol in scene.state_symbols:
        q, dq = state_map[symbol]
        q += dq * dt
        dq += ddq[symbol] * dt
        new_state_map[symbol] = [q, dq]
    return new_state_map

In [ ]:
max_time = 10000

p1 = Connector() | Pendulum(mass=1.1)
p2 = p1 | Pendulum(mass=1.2)
p3 = p2 | Pendulum()
scene = Scene([p2])

state_map = {
    p1.spec.theta: [0., 0.],
    p2.spec.theta: [0., 1.],
    p3.spec.theta: [1.3, 0.],
}
state_maps = [state_map]
for i in range(max_time):
    state_map = tick(scene, state_map)
    state_maps.append(state_map)
    
None

In [ ]:
@interact(t=(0, max_time))
def f(t=0):
    fig, ax = plt.subplots(figsize=(12,6))
    ax.set_aspect('equal')
    ax.set_xlim(-5., 5.)
    ax.set_ylim(-5., 5.)

    state_map = state_maps[t]
    theta1 = state_map[p1.spec.theta][0]
    theta2 = state_map[p2.spec.theta][0]
    #theta3 = state_map[p3.spec.theta][0]

    x1 = np.sin(theta1)
    y1 = -np.cos(theta1)
    x2 = x1 + np.sin(theta2)
    y2 = y1 - np.cos(theta2)
    #x3 = x2 + np.sin(theta3)
    #y3 = y2 - np.cos(theta3)
    
    draw_line(ax, 0., 0., x1, y1)
    draw_circle(ax, x1, y1, 0.2)
    draw_line(ax, x1, y1, x2, y2)
    draw_circle(ax, x2, y2, 0.2)
    #draw_line(ax, x2, y2, x3, y3)
    #draw_circle(ax, x3, y3, 0.2)

In [ ]:
scene.get_pert_accel_coefficient(scene.connectors[0], scene.connectors[1])